In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv('df_trans_train.csv')

In [8]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

## estandarizacion

In [11]:
from sklearn.preprocessing import StandardScaler
#normalizamos los datos
scaler=StandardScaler()
scaler.fit(df) # calculo la media para poder hacer la transformacion
X_scaled=scaler.transform(df)# Ahora si, escalo los datos y los normalizo

## Redusco dimencion con PCA

In [12]:
from sklearn.decomposition import PCA

pca = PCA(n_components=45)
X_nuevo = pca.fit_transform(X_scaled) 

In [13]:
print("shape of X_pca", X_nuevo.shape)
expl = pca.explained_variance_ratio_
print(expl)
print('suma:',sum(expl[0:29]))

shape of X_pca (410000, 45)
[1.40075546e-01 6.22584530e-02 5.14719744e-02 4.83107878e-02
 4.68139614e-02 4.30828788e-02 4.14193375e-02 3.32533203e-02
 3.21680702e-02 2.88464548e-02 2.85140454e-02 2.79212322e-02
 2.69824225e-02 2.58167422e-02 2.49803242e-02 2.46071960e-02
 2.40155299e-02 2.29727799e-02 2.24365921e-02 2.19360679e-02
 2.19115991e-02 2.17905528e-02 2.12956448e-02 2.07977512e-02
 2.02959865e-02 1.97834623e-02 1.81537194e-02 1.73145424e-02
 1.53135598e-02 1.32344207e-02 1.09869138e-02 9.06827830e-03
 4.42585042e-03 4.08339268e-03 3.66060935e-03 2.06020706e-27
 8.93458020e-28 1.90238567e-28 1.40086014e-28 7.34440389e-29
 4.37406860e-29 3.39612083e-29 2.65119867e-29 7.59807472e-30
 1.09677837e-30]
suma: 0.9545405347283406


In [14]:
pca = PCA(n_components=29)
X_nuevo = pca.fit_transform(X_scaled)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_nuevo, y, test_size=0.30, random_state=42)

### Benchmark

In [ ]:
print('El benchmark es:',y[y == 1].shape[0]/y.shape[0])

EL benchmark es: 0.6229658536585366


### Decision Tree

In [ ]:
from sklearn.metrics import accuracy_score 

# Predecimos

y_pred = clf.predict(X_test)

# Evaluamos

print('El score del modelo es:', accuracy_score(y_test, y_pred))

In [18]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=42) 
model = GridSearchCV(cv=5, error_score= np.nan,
             estimator=clf,
             param_grid={'ccp_alpha': [0.1, 0.01, 0.001],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 6, 7, 8, 9],
                         'max_features': ['auto', 'sqrt', 'log2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [19]:
model.fit(X_train, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(
c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'ccp_alpha': [0.1, 0.01, 0.001],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 6, 7, 8, 9],
                         'max_features': ['auto', 'sqrt', 'log2']},
             verbose=True)

In [22]:
model.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.001, criterion='entropy', max_depth=9,
                       max_features='auto', random_state=42)

In [23]:
print("Mejores hiperparámetros: "+str(model.best_params_))
print("Mejor Score: "+str(model.best_score_)+'\n')

Mejores hiperparámetros: {'ccp_alpha': 0.001, 'criterion': 'entropy', 'max_depth': 9, 'max_features': 'auto'}
Mejor Score: 0.9795783972125436



In [28]:
clf = DecisionTreeClassifier(ccp_alpha= 0.001, criterion='entropy',max_depth = 9 , max_features= 'auto',random_state=42) 

In [30]:
clf.fit(X_train, y_train)

c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\tree\_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


DecisionTreeClassifier(ccp_alpha=0.001, criterion='entropy', max_depth=9,
                       max_features='auto', random_state=42)

In [32]:
y_pred = clf.predict(X_test)

In [33]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_test,y_pred))

0.9821707317073171


In [35]:
from sklearn.metrics import recall_score
precision = recall_score(y_test, y_pred, average= None)
print(precision)

[0.97640829 0.98566819]


In [36]:
df_test = pd.read_csv('df_trans_test.csv')

In [39]:
from sklearn.preprocessing import StandardScaler
#normalizamos los datos
scaler=StandardScaler()
scaler.fit(df_test) # calculo la media para poder hacer la transformacion
X_to_pred=scaler.transform(df_test)# Ahora si, escalo los datos y los normalizo

## Redusco dimencion con PCA

In [40]:
from sklearn.decomposition import PCA

pca = PCA(n_components=29)
X_df_test = pca.fit_transform(X_to_pred) 

In [41]:
pred = clf.predict(X_df_test)

In [42]:
pred = pd.DataFrame()

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

## Booosting

In [24]:
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier

In [25]:
clf_xgb = xgb.XGBClassifier(n_estimators=150,
                            seed=42, use_label_encoder= False)
                            
# Entrenamos
clf_xgb.fit(X_train, y_train)

c:\Users\Carlos\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=150,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42, ...)

In [26]:
import sklearn.metrics as metrics

y_train_pred = clf_xgb.predict(X_train)
y_test_pred = clf_xgb.predict(X_test)

# Evaluamos
print('Accuracy en train:', metrics.accuracy_score(y_train, y_train_pred))
print('Accuracy en test:', metrics.accuracy_score(y_test, y_test_pred))

Accuracy en train: 1.0
Accuracy en test: 0.9999674796747967
